In [25]:
from bs4 import BeautifulSoup
import requests
import time

def cnn_news_scarping(topic):
    url = 'https://www.cnn.co.jp'
    r = requests.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    
    #クラス名が変わるので注意！！
    elements = soup.select('.pg-container li a')
    elements = elements[6:11] + [elements[14], elements[16]]
    
    topic = topic
    #トピックのURL取得
    for i, element in enumerate(elements):
        if topic in element.text:
            categories_num = i
    
    element = elements[categories_num]
    
    topic_url = element['href']
    URL = url + topic_url
    
    r = requests.get(URL)
    soup = BeautifulSoup(r.text, "html.parser")
    
    #クラス名が変わるので注意！！
    elements = soup.select('.cd a')
    
    elements = elements[:3]
    
    news_urls = []
    news_titles = []
    #ニュースのURLとタイトルを取得
    for element in elements:
        news_url =  url + element['href']
        news_title = element.text
    
        news_urls.append(news_url)
        news_titles.append(news_title)
    
    news_text = []
    #ニュースの本文を取得
    for i, news_url in enumerate(news_urls):
        text = ""
        
        for j in range(1, 7):
            num = str(j)
            page = '-' + num + '.html'    
            # 文字列の置換を行い新しいURLを生成
            news_url_page = news_urls[i].replace('.html', page)
    
            #ページ数が複数のとき、ページ遷移し本文を取得
            response = requests.get(news_url_page)
    
            # レスポンスのステータスコードを確認
            if response.status_code != 404: 
                r = requests.get(news_url_page)
                soup = BeautifulSoup(r.text, "html.parser")
                #クラス名が変わるので注意！！ 
                elements = soup.find(id = 'leaf-body')
                text += elements.text
            else:
                continue
    
        news_text.append(text)

    articles = []
    for url, title, text in zip(news_urls, news_titles, news_text):
        article = {
            "title" : title,
            "URL" : url,
            "text" : text
        }
        articles.append(article)

    return articles

In [24]:
def clean_data(articles):
    for i in range(len(articles)):
        articles[i]['title'] = articles[i]['title'].replace('\u3000', '') 
        articles[i]['title'] = articles[i]['title'].lstrip('0123456789')
        articles[i]['text'] = articles[i]['text'].replace('\u3000', '')
        articles[i]['text'] = articles[i]['text'].replace('\n', '')

    return articles

In [28]:
a = cnn_news_scarping('USA')

In [30]:
a = clean_data(a)